# Creating a naive neural network to model a logic implication

## Creating and exporting a dataset

In [3]:
import pandas as pd
import json
import numpy as np
from typing import List

In [4]:
data_set = np.array(
    [
        {
            "X1": 0,
            "X2": 0,
            "Y": 1,
        },
        {
            "X1": 0,
            "X2": 1,
            "Y": 1,
        },
        {
            "X1": 1,
            "X2": 0,
            "Y": 0,
        },
        {
            "X1": 1,
            "X2": 1,
            "Y": 1,
        },
    ],
    dtype=dict,
)

In [5]:
database = json.dumps(data_set.tolist())
try:
    database_json_file = pd.read_json(database)
    database_json_file.to_csv("./data/logic.csv", encoding="utf-8")
except:
    raise "It's not possible to create logic.csv"

## Creating a perceptron class

In [6]:
class Perceptron(object):
    def __init__(self, n_of_inputs: int) -> None:
        self.weights = np.zeros(n_of_inputs)
        self.bias = 0

    def set_weights(self, weights: list) -> None:
        if len(weights) != len(self.weights):
            raise "Impossible to set weight list. The len are incompatible"
        for i in range(len(self.weights)):
            self.weights[i] = weights[i]

    def set_bias(self, value: np.float64) -> None:
        self.bias = value

    def predict(self, inputs: list):
        summation = np.dot(inputs, self.weights) + self.bias
        if summation <= 0:
            return 0
        return 1

## Getting input values

In [7]:
df = pd.read_csv("./data/logic.csv")

## Setting a neural network architecture

In [15]:
P1 = Perceptron(n_of_inputs=2)
P1.set_bias(2)
P1.set_weights([2, 2])

In [16]:
P2 = Perceptron(n_of_inputs=2)
P2.set_bias(0.7)
P2.set_weights([-1, 2])

In [17]:
output = Perceptron(n_of_inputs=2)
output.set_bias(-1.5)
output.set_weights([1, 1])

## Getting neural network final output

In [18]:
def getting_output_value(
    output_perceptron: Perceptron,
    perceptrons: List[Perceptron],
    df: pd.DataFrame,
) -> list:
    output_value = []
    inputs_data = df.loc[:, ["X1", "X2"]]
    columns_data = inputs_data.columns

    for i in range(len(inputs_data)):
        for j in range(len(columns_data) - 1):
            x1 = inputs_data[columns_data[j]][i]
            x2 = inputs_data[columns_data[j + 1]][i]
            response = output_perceptron.predict(
                [perceptrons[j].predict([x1, x2]), perceptrons[j + 1].predict([x1, x2])]
            )
            output_value.append(response)
    return output_value

## Exporting predict values got by output percetron

In [19]:
output_list = getting_output_value(output, [P1, P2], df)
df["Ŷ"] = np.array(output_list, dtype=np.int64)

## Exporting the loss function $ Y - \hat{Y}$

In [20]:
df["e"] = pd.Series(
    np.array([df["Y"][i] - df["Ŷ"][i] for i in range(len(df["Y"]))], dtype=np.int64)
)

## Exporting dataframe as csv file

In [21]:
df.to_csv("./data/logic.csv", encoding="utf-8")